<a href="https://colab.research.google.com/github/darshank528/Artificial-Neural-Network/blob/master/ResNet34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn

#setting device configuration
device=torch.device("cuda" if torch.cuda.is_available else "cpu")

def ResNet34():
  class Identity_block(nn.Module):
    #identity block
    def __init__(self,in_channel,out_channel):
      super(Identity_block,self).__init__()
      self.convib = nn.Conv2d(in_channel,out_channel,3,1,1)
      self.bnib = nn.BatchNorm2d(out_channel)

    def forward(self,x):
      x_shortcut = x    
      x = self.convib(x)
      x = self.bnib(x)
      x = torch.relu(x)
      x = self.convib(x)
      x = self.bnib(x)
      x = torch.relu(x)
      x = self.convib(x)
      x = self.bnib(x)
      #creating a shortcut connection
      x = x + x_shortcut
      x = torch.relu(x)
      return x

  class Convolutional_block(nn.Module):
    #convolution block
    def __init__(self,in_channel,out_channel):
      super(Convolutional_block,self).__init__()
      self.convcb1 = nn.Conv2d(in_channel,out_channel,3,2,1)
      self.convcb2 = nn.Conv2d(out_channel,out_channel,3,1,1)
      self.bncb = nn.BatchNorm2d(out_channel)

    def forward(self,x):
      x_shortcut = x
      x = self.convcb1(x)
      x = self.bncb(x)
      x = torch.relu(x)
      x = self.convcb2(x)
      x = self.bncb(x)
      x = torch.relu(x)
      x = self.convcb2(x)
      x = self.bncb(x)
      x_shortcut = self.convcb1(x_shortcut)
      x_shortcut = self.bncb(x_shortcut)
      #creating a shortcut connection
      x = x + x_shortcut
      x = torch.relu(x)
      return x


  class ResNet(nn.Module):
    #main resnet model
    def __init__(self):
      super(ResNet,self).__init__()
      self.conv = nn.Conv2d(3,64,7,2,3)
      self.pool1 = nn.MaxPool2d(3,2,1)
      self.pool2 = nn.AvgPool2d(3,2,0)
      self.fc1 = nn.Linear(4608,1000)
      self.fc2 = nn.Linear(1000,10)    

    def forward(self,x):
      #initial covolution and pooling layers
      x = self.conv(x)
      x = self.pool1(x)

      #stage1--passed through 3 identity blocks
      model_ib1 = Identity_block(64,64).to(device)
      x = model_ib1(x)
      x = model_ib1(x)
      x = model_ib1(x)

      #stage2--passed through 1 convolution block for downsampling and 3 identity blocks
      model_ib2 = Identity_block(128,128).to(device)
      model_cb1 = Convolutional_block(64,128).to(device)
      x = model_cb1(x)
      x = model_ib2(x)
      x = model_ib2(x)
      x = model_ib2(x)

      #stage3--passed through 1 convolution block for downsampling and 5 identity blocks
      model_ib3 = Identity_block(256,256).to(device)
      model_cb2 = Convolutional_block(128,256).to(device)
      x = model_cb2(x)
      x = model_ib3(x)
      x = model_ib3(x)
      x = model_ib3(x)
      x = model_ib3(x)
      x = model_ib3(x)

      #stage4--passed through 1 convolution block for downsampling and 2 identity blocks
      model_ib4 = Identity_block(512,512).to(device)
      model_cb3 = Convolutional_block(256,512).to(device)
      x = model_cb3(x)
      x = model_ib4(x)
      x = model_ib4(x)

      #final pooling and fully-connected layers
      x = self.pool2(x)
      x = x.view(-1,3*3*512)
      x = torch.relu(self.fc1(x))
      x = self.fc2(x)

      return x

  #creating an object of resnet model and pushing it to device(CPU/GPU)
  model = ResNet().to(device)
  return model